In [ ]:
#Want to pull out restaurant type, then One-Hot Encode that into df

In [101]:
#imports
import numpy as np
import pandas as pd
import re

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os


### Load Chrome webdriver

In [115]:
#from selenium.webdriver.common.action_chains import ActionChains

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)

### Restaurant Data Table

In [76]:
#import table with restaurant data
restaurant_info = pd.read_pickle('./restaurant_big.pkl')

In [77]:
restaurant_info.head()

,Business Account Number,DBA Name,Street Address,Source Zipcode,Currently Open,Years Open,Year Opened,Month Opened,Year Closed,Success,Zip Code,Neighborhood,ZHVI_AllHomes,Year,Month
3340,1002775,Novy,4000 24 Th St,94114,1.0,4.265753,2015,2,None,1,94114.0,Castro/Noe Valley,1463000.0,2015.0,2.0
3535,1007372,Liholiho Yacht Club,871 Sutter St,94109,1.0,4.347945,2015,1,None,1,94109.0,Polk/Russian Hill (Nob Hill),985900.0,2015.0,1.0
3536,1007387,Henry's Hungry,1244 Castro St,94114,1.0,4.347945,2015,1,None,1,94114.0,Castro/Noe Valley,1449400.0,2015.0,1.0
3537,1007392,Hillside Supper Club,300 Precita Ave,94110,1.0,4.347945,2015,1,None,1,94110.0,Inner Mission/Bernal Heights,1140000.0,2015.0,1.0
3539,1007450,Candicake,2124 Folsom St,94110,1.0,4.339726,2015,1,None,1,94110.0,Inner Mission/Bernal Heights,1140000.0,2015.0,1.0


### Scrape google review rating, price point, and restaurant description from google

In [116]:
def getRestaurantInfo(series):
    name = series[0]
    address = series[1]

    google_url = 'http://www.google.com/'
    driver.get(google_url)

    query = driver.find_element_by_name('q')
    query.clear()
    query.send_keys(name + ' ' + address) 
    query.send_keys(Keys.RETURN)
    
    # get RATING
    try:
        rating = driver.find_element_by_class_name('rtng').text
    except:
        rating = 0
        
    # get PRICE POINT (number of dollar-signs)
    try:
        price = driver.find_element_by_class_name('YhemCb').text
    except:
        price = 0
        
    # get RESTAURANT TYPE
    try:
        desc = driver.find_element_by_xpath(
            '//*[@id="rhs_block"]/div/div[1]/div/div[1]/div[2]/div[1]/div[2]/div[2]/div[3]/div/span[2]'
        ).text
    except:
        desc = 0
    
    if desc == 0:
        try:
            desc = driver.find_element_by_xpath(
                '//*[@id="rhs_block"]/div/div[1]/div/div[1]/div[2]/div[4]/div/span[2]'
            ).text
        except:
            desc = 0
            
    time.sleep(5)   
    return rating, price, desc


**First iteration of the scraper**

In [79]:
#Run web scraper and save data to new column, 'google_results'
#restaurant_info['google_results'] = restaurant_info[['DBA Name', 'Street Address']].apply(getRestaurantInfo, axis=1)

In [80]:
restaurant_info.head()

,Business Account Number,DBA Name,Street Address,Source Zipcode,Currently Open,Years Open,Year Opened,Month Opened,Year Closed,Success,Zip Code,Neighborhood,ZHVI_AllHomes,Year,Month,google_results
3340,1002775,Novy,4000 24 Th St,94114,1.0,4.265753,2015,2,None,1,94114.0,Castro/Noe Valley,1463000.0,2015.0,2.0,"(4.2, 2, Greek restaurant)"
3535,1007372,Liholiho Yacht Club,871 Sutter St,94109,1.0,4.347945,2015,1,None,1,94109.0,Polk/Russian Hill (Nob Hill),985900.0,2015.0,1.0,"(4.6, 3, Californian restaurant)"
3536,1007387,Henry's Hungry,1244 Castro St,94114,1.0,4.347945,2015,1,None,1,94114.0,Castro/Noe Valley,1449400.0,2015.0,1.0,"(0, 0, 0)"
3537,1007392,Hillside Supper Club,300 Precita Ave,94110,1.0,4.347945,2015,1,None,1,94110.0,Inner Mission/Bernal Heights,1140000.0,2015.0,1.0,"(4.4, 2, New American restaurant)"
3539,1007450,Candicake,2124 Folsom St,94110,1.0,4.339726,2015,1,None,1,94110.0,Inner Mission/Bernal Heights,1140000.0,2015.0,1.0,"(0, 0, 0)"


In [117]:
#Check to see how much info was able to be scraped
restaurant_info = restaurant_info.loc[restaurant_info['google_results'] != (0, 0, 0)]
restaurant_info.shape

(1540, 19)

**Had to change price from len(text) to the text itself, since sometimes the description of the restaurant was located where the price should have been, so the lengths came out all wonky.**

In [118]:
#Re-run web scraper on list of restaurants that were able to be scraped,
#and overwrite data to new column, 'google_results' 
restaurant_info['google_results'] = restaurant_info[['DBA Name', 'Street Address']].apply(getRestaurantInfo, axis=1)

In [119]:
#Pickle dataset
restaurant_info.to_pickle('./restaurant_scrape.pkl')

### Clean Dataset

In [120]:
restaurant_info = pd.read_pickle('./restaurant_scrape.pkl')

In [121]:
#Create 'Rating' column
restaurant_info['Rating'] = restaurant_info['google_results'].apply(lambda x: x[0])

In [122]:
#Create 'Price' column --> Ordinal data (should be between 1-4)
restaurant_info['Price'] = restaurant_info['google_results'].apply(lambda x: x[1])

In [123]:
restaurant_info['Food'] = restaurant_info['google_results'].apply(lambda x: x[2])

In [124]:
restaurant_info.head()

,Business Account Number,DBA Name,Street Address,Source Zipcode,Currently Open,Years Open,Year Opened,Month Opened,Year Closed,Success,Zip Code,Neighborhood,ZHVI_AllHomes,Year,Month,google_results,Rating,Price,Food
3340,1002775,Novy,4000 24 Th St,94114,1.0,4.265753,2015,2,None,1,94114.0,Castro/Noe Valley,1463000.0,2015.0,2.0,"(4.2, $$, Greek restaurant)",4.2,$$,Greek restaurant
3535,1007372,Liholiho Yacht Club,871 Sutter St,94109,1.0,4.347945,2015,1,None,1,94109.0,Polk/Russian Hill (Nob Hill),985900.0,2015.0,1.0,"(4.6, $$$, Californian restaurant)",4.6,$$$,Californian restaurant
3537,1007392,Hillside Supper Club,300 Precita Ave,94110,1.0,4.347945,2015,1,None,1,94110.0,Inner Mission/Bernal Heights,1140000.0,2015.0,1.0,"(4.4, $$, New American restaurant)",4.4,$$,New American restaurant
3540,1007473,The Pin Up,772 Folsom St,94108,1.0,4.350685,2015,1,None,1,94108.0,Chinatown,852600.0,2015.0,1.0,"(4.0, Diner, 0)",4.0,Diner,0
3541,1007479,Belle Cora,565 Green St,94133,1.0,4.336986,2015,1,None,1,94133.0,North Beach/Chinatown,1189000.0,2015.0,1.0,"(4.6, $$, Bar)",4.6,$$,Bar


In [125]:
set(restaurant_info['Rating'])

{'',
 0,
 '1.0',
 '1.7',
 '2.3',
 '2.4',
 '2.5',
 '2.6',
 '2.7',
 '2.8',
 '2.9',
 '3.0',
 '3.1',
 '3.2',
 '3.3',
 '3.4',
 '3.5',
 '3.6',
 '3.7',
 '3.8',
 '3.9',
 '4.0',
 '4.1',
 '4.2',
 '4.3',
 '4.4',
 '4.5',
 '4.6',
 '4.7',
 '4.8',
 '4.9',
 '5.0'}

In [126]:
set(restaurant_info['Price'])

{'$',
 '$$',
 '$$$',
 '$$$$',
 0,
 '2-star hotel',
 '3-star hotel',
 '4-star hotel',
 '5-star hotel',
 'American restaurant',
 'Apartment building in San Francisco, California',
 'Art center in San Francisco, California',
 'Artist in San Francisco, California',
 'Arts organization',
 'Asian fusion restaurant',
 'Asian restaurant',
 'Association or organization',
 'Auditorium in San Francisco, California',
 'Authentic Japanese restaurant',
 'Bakery',
 'Bar',
 'Bistro',
 'Boat tour agency in San Francisco, California',
 'Bubble tea store',
 'Burmese restaurant',
 'Burrito restaurant',
 'Bus station in South San Francisco, California',
 'Butcher shop',
 'Cafe',
 'Cafeteria',
 'Cake shop',
 'Caterer',
 'Catering food and drink supplier',
 'Cell phone store in San Francisco, California',
 'Chicken restaurant',
 'Chinese restaurant',
 'Chocolate shop',
 'Coffee shop',
 'Colombian restaurant',
 'Convenience store',
 'Cookie shop',
 'Corporate office',
 'Courthouse in San Francisco, California

In [127]:
set(restaurant_info['Food'])

{0,
 '0.7 mi',
 '1.0 mi',
 '1.1 mi',
 '1.7 mi',
 '2.2 mi',
 '2.5 mi',
 '4.3 mi',
 'Afghani restaurant',
 'American restaurant',
 'Asian fusion restaurant',
 'Asian restaurant',
 'Bagel shop',
 'Bakery',
 'Bar',
 'Bar & grill',
 'Barbecue restaurant',
 'Bistro',
 'Brazilian restaurant',
 'Breakfast restaurant',
 'Brunch restaurant',
 'Bubble tea store',
 'Burmese restaurant',
 'Cafe',
 'Cajun restaurant',
 'Californian restaurant',
 'Cantonese restaurant',
 'Chicken restaurant',
 'Chicken wings restaurant',
 'Chinese noodle restaurant',
 'Chinese restaurant',
 'Chocolate shop',
 'Cocktail bar',
 'Coffee shop',
 'Cookie shop',
 'Crêperie',
 'Cuban restaurant',
 'Deli',
 'Diner',
 'Dinner theater',
 'Donut shop',
 'Dumpling restaurant',
 'Eclectic restaurant',
 'Eritrean restaurant',
 'Espresso Bar',
 'Event venue in San Francisco, California',
 'Fast food restaurant',
 'Filipino restaurant',
 'Fish & chips restaurant',
 'Food Court',
 'French restaurant',
 'Fusion restaurant',
 'Gastropu

In [136]:
# Need to locate rows where 'Price' is not 0 or does not include '$', then
# move the values from 'Price' to 'Food'

rows_with_misplaced_desc = restaurant_info.loc[restaurant_info['Price'] != 0].loc[
    restaurant_info['Price'] != '$'].loc[
    restaurant_info['Price'] != '$$'].loc[
    restaurant_info['Price'] != '$$$'].loc[
    restaurant_info['Price'] != '$$$$']

In [154]:
rows_with_misplaced_desc.shape

(496, 19)

In [155]:
rows_with_misplaced_desc['Food'] = rows_with_misplaced_desc['Price'].apply(lambda x: x)

3540                                                Diner
3567                                   Takeout Restaurant
3573                    Farm in San Francisco, California
3585                                   Chinese restaurant
3601                                    Korean restaurant
3615                                   Mexican restaurant
3621                                  Delivery Restaurant
3622                            Middle Eastern restaurant
3633                                          Coffee shop
3636                                    Arts organization
3637                                    Arts organization
3638                                       Chocolate shop
3639                                     Pizza restaurant
3657                           Korean barbecue restaurant
3662                                   Mexican restaurant
3678                                               Bakery
3680                                          Coffee shop
3705          

In [156]:
rows_with_misplaced_desc['Price'] = 0

In [157]:
rows_with_misplaced_desc.head()

,Business Account Number,DBA Name,Street Address,Source Zipcode,Currently Open,Years Open,Year Opened,Month Opened,Year Closed,Success,Zip Code,Neighborhood,ZHVI_AllHomes,Year,Month,google_results,Rating,Price,Food
3540,1007473,The Pin Up,772 Folsom St,94108,1.0,4.350685,2015,1,None,1,94108.0,Chinatown,852600.0,2015.0,1.0,"(4.0, Diner, 0)",4.0,0,Diner
3567,1007779,Go Jojo's Mojo,55 Duboce Ave,94103,1.0,4.334247,2015,1,None,1,94103.0,South of Market,885100.0,2015.0,1.0,"(3.8, Takeout Restaurant, 0)",3.8,0,Takeout Restaurant
3573,1007919,Growing Up Farms,5801 Mission St,94112,1.0,4.315068,2015,1,None,1,94112.0,Ingelside-Excelsior/Crocker-Amazon,753200.0,2015.0,1.0,"(0, Farm in San Francisco, California, 0)",0,0,"Farm in San Francisco, California"
3585,1008176,Little Beijing Restaurant,1316 Noriega St,94122,1.0,4.279452,2015,1,None,1,94122.0,Sunset,986600.0,2015.0,1.0,"(4.1, Chinese restaurant, 0)",4.1,0,Chinese restaurant
3601,1008549,Wang Dae Bak,2642 Ocean Ave,94132,1.0,4.263014,2015,2,None,1,94132.0,Lake Merced,885900.0,2015.0,2.0,"(4.1, Korean restaurant, 0)",4.1,0,Korean restaurant


In [139]:
# Locations without Price or Food Type Data
#to_exclude = restaurant_info.loc[restaurant_info['Price'] == 0]


In [149]:
# Cheap Restaurants
cheap_restaurants = restaurant_info.loc[restaurant_info['Price'] == '$']
cheap_restaurants.shape

(454, 19)

In [148]:
# Semi-Expensive Restaurants
semi_expensive_restaurants = restaurant_info.loc[restaurant_info['Price'] == '$$']
semi_expensive_restaurants.shape

(451, 19)

In [152]:
# Expensive Restaurants
expensive_restaurants = restaurant_info.loc[restaurant_info['Price'] == '$$$']
expensive_restaurants.shape

(49, 19)

In [153]:
# Super Expensive Restaurants
super_expensive_restaurants = restaurant_info.loc[restaurant_info['Price'] == '$$$$']
super_expensive_restaurants.shape

(10, 19)

### Put small tables back together

In [238]:
restaurants_cleaner = pd.concat([rows_with_misplaced_desc, 
                                 cheap_restaurants, 
                                 semi_expensive_restaurants, 
                                 expensive_restaurants, 
                                 super_expensive_restaurants])

In [239]:
set(restaurants_cleaner['Food'])

{0,
 '2-star hotel',
 '3-star hotel',
 '4-star hotel',
 '5-star hotel',
 'Afghani restaurant',
 'American restaurant',
 'Apartment building in San Francisco, California',
 'Art center in San Francisco, California',
 'Artist in San Francisco, California',
 'Arts organization',
 'Asian fusion restaurant',
 'Asian restaurant',
 'Association or organization',
 'Auditorium in San Francisco, California',
 'Authentic Japanese restaurant',
 'Bagel shop',
 'Bakery',
 'Bar',
 'Bar & grill',
 'Barbecue restaurant',
 'Bistro',
 'Boat tour agency in San Francisco, California',
 'Brazilian restaurant',
 'Breakfast restaurant',
 'Brunch restaurant',
 'Bubble tea store',
 'Burmese restaurant',
 'Burrito restaurant',
 'Bus station in South San Francisco, California',
 'Butcher shop',
 'Cafe',
 'Cafeteria',
 'Cajun restaurant',
 'Cake shop',
 'Californian restaurant',
 'Cantonese restaurant',
 'Caterer',
 'Catering food and drink supplier',
 'Cell phone store in San Francisco, California',
 'Chicken res

In [240]:
restaurants_cleaner = restaurants_cleaner.loc[
    restaurants_cleaner.Food != '2-star hotel'].loc[
    restaurants_cleaner.Food != '3-star hotel'].loc[
    restaurants_cleaner.Food != '4-star hotel'].loc[
    restaurants_cleaner.Food != '5-star hotel'].loc[
    restaurants_cleaner.Food != 'Apartment building in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Art center in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Artist in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Arts organization'].loc[
    restaurants_cleaner.Food != 'Association or organization'].loc[
    restaurants_cleaner.Food != 'Auditorium in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Boat tour agency in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Bus station in South San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Butcher shop'].loc[
    restaurants_cleaner.Food != 'Cafeteria'].loc[
    restaurants_cleaner.Food != 'Caterer'].loc[
    restaurants_cleaner.Food != 'Catering food and drink supplier'].loc[
    restaurants_cleaner.Food != 'Cell phone store in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Convenience store'].loc[
    restaurants_cleaner.Food != 'Corporate office'].loc[
    restaurants_cleaner.Food != 'Courthouse in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Culinary school in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Delivery Restaurant'].loc[
    restaurants_cleaner.Food != 'Education in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Event management company in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Event venue in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Farm in San Francisco, California'].loc[
    restaurants_cleaner.Food != "Farmers' market in San Francisco, California"].loc[
    restaurants_cleaner.Food != 'Federal government office in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Food Court'].loc[
    restaurants_cleaner.Food != 'Food products supplier'].loc[
    restaurants_cleaner.Food != 'Forklift dealer in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Gift shop in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Grocery store'].loc[
    restaurants_cleaner.Food != 'Health food store'].loc[
    restaurants_cleaner.Food != 'Interior designer in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Internet cafe in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Market'].loc[
    restaurants_cleaner.Food != 'Mobile caterer'].loc[
    restaurants_cleaner.Food != 'Money transfer service in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Non-profit organization in San Francisco, California'].loc[
    restaurants_cleaner.Food !=  'Park in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Point of interest'].loc[
    restaurants_cleaner.Food != 'Pool hall in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Popcorn store'].loc[
    restaurants_cleaner.Food != 'Post office in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Private school in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Private university in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Produce market'].loc[
    restaurants_cleaner.Food != 'Produce wholesaler'].loc[
    restaurants_cleaner.Food != 'Professional services in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Public library in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Publisher in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Restaurant supply store in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Retirement community in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'School in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Self-Catering Accommodation'].loc[
    restaurants_cleaner.Food != 'Shipyard in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Shopping center in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Shopping mall'].loc[
    restaurants_cleaner.Food != 'Shopping mall in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Snack bar'].loc[
    restaurants_cleaner.Food != 'Soft drinks shop'].loc[
    restaurants_cleaner.Food != 'Supermarket'].loc[
    restaurants_cleaner.Food != 'Takeout Restaurant'].loc[
    restaurants_cleaner.Food != 'Tobacco shop in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'University library in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Wedding bakery'].loc[
    restaurants_cleaner.Food != 'Wholesale bakery'].loc[
    restaurants_cleaner.Food != 'Wholesale grocer'].loc[
    restaurants_cleaner.Food != 'Wholesaler in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Wine store'].loc[
    restaurants_cleaner.Food != 'Flooring contractor in San Francisco, California'].loc[
    restaurants_cleaner.Food != 'Seafood wholesaler']


In [241]:
restaurants_cleaner.shape

(1285, 19)

In [242]:
restaurants_cleaner.loc[restaurants_cleaner['Success'] == 0].shape

(350, 19)

In [243]:
set(restaurants_cleaner['Food'])

{0,
 'Afghani restaurant',
 'American restaurant',
 'Asian fusion restaurant',
 'Asian restaurant',
 'Authentic Japanese restaurant',
 'Bagel shop',
 'Bakery',
 'Bar',
 'Bar & grill',
 'Barbecue restaurant',
 'Bistro',
 'Brazilian restaurant',
 'Breakfast restaurant',
 'Brunch restaurant',
 'Bubble tea store',
 'Burmese restaurant',
 'Burrito restaurant',
 'Cafe',
 'Cajun restaurant',
 'Cake shop',
 'Californian restaurant',
 'Cantonese restaurant',
 'Chicken restaurant',
 'Chicken wings restaurant',
 'Chinese noodle restaurant',
 'Chinese restaurant',
 'Chocolate shop',
 'Cocktail bar',
 'Coffee shop',
 'Colombian restaurant',
 'Cookie shop',
 'Crêperie',
 'Cuban restaurant',
 'Deli',
 'Dessert restaurant',
 'Dessert shop',
 'Diner',
 'Dinner theater',
 'Donut shop',
 'Dumpling restaurant',
 'Eclectic restaurant',
 'Eritrean restaurant',
 'Espresso Bar',
 'Fast food restaurant',
 'Filipino restaurant',
 'Fish & chips restaurant',
 'French restaurant',
 'Frozen yogurt shop',
 'Fusion r

In [244]:
restaurants_cleaner['Food'].replace([
    'Afghani restaurant', 
    'Halal restaurant', 
    'Lebanese restaurant', 
    'Middle Eastern restaurant', 
    'Moroccan restaurant', 
    'Persian restaurant',  
    'Turkish restaurant', 
    'Yemenite restaurant'], 'Middle Eastern', inplace = True)

restaurants_cleaner['Food'].replace([
    'American restaurant', 
    'New American restaurant', 
    'Californian restaurant'], 'American', inplace = True)

restaurants_cleaner['Food'].replace([
    'Asian fusion restaurant', 
    'Asian restaurant', 
    'Dumpling restaurant'], 'Asian', inplace = True)

restaurants_cleaner['Food'].replace([
    'Bar', 
    'Bar & grill', 
    'Cocktail bar', 
    'Gastropub', 
    'Irish pub', 
    'Jazz club', 
    'Live music venue', 
    'Lounge', 
    'Night club', 
    'Night club in San Francisco, California', 
    'Sports bar', 
    'Wine bar'], 'Bar', inplace = True)

restaurants_cleaner['Food'].replace([
    'Bakery', 
    'Cake shop', 
    'Chocolate shop', 
    'Cookie shop',
    'Crêperie', 
    'Dessert restaurant', 
    'Dessert shop', 
    'Donut shop', 
    'Frozen yogurt shop', 
    'Ice cream shop', 
    'Pastry shop', 
    'Patisserie', 
    'Pie shop'], 'Bakery/Desserts', inplace = True)

restaurants_cleaner['Food'].replace([
    'Cafe', 
    'Coffee shop', 
    'Espresso Bar', 
    'Tea house', 
    'Bubble tea store'], 'Coffee/Tea/Cafe', inplace = True) 

restaurants_cleaner['Food'].replace([
    'Cantonese restaurant', 
    'Chinese noodle restaurant',
    'Chinese restaurant', 
    'Hong Kong style fast food restaurant', 
    'Hot pot restaurant', 
    'Hunan restaurant', 
    'Noodle shop', 
    'Shanghainese restaurant', 
    'Sichuan restaurant', 
    'Taiwanese restaurant'], 'Chinese', inplace = True)

restaurants_cleaner['Food'].replace([
    'Burmese restaurant', 
    'Laotian restaurant', 
    'Southeast Asian restaurant', 
    'Pho restaurant', 
    'Thai restaurant', 
    'Filipino restaurant', 
    'Vietnamese restaurant'], 'Southeast Asian', inplace = True)

restaurants_cleaner['Food'].replace([
    'Breakfast restaurant', 
    'Brunch restaurant', 
    'Bagel shop'], 'Breakfast/Brunch', inplace = True)

restaurants_cleaner['Food'].replace([
    'Authentic Japanese restaurant', 
    'Izakaya restaurant', 
    'Japanese restaurant', 
    'Ramen restaurant', 
    'Sushi restaurant', 
    'Udon noodle restaurant',], 'Japanese', inplace = True)

restaurants_cleaner['Food'].replace([
    'Korean barbecue restaurant', 
    'Korean restaurant'], 'Korean', inplace = True)

restaurants_cleaner['Food'].replace([
    'Brazilian restaurant', 
    'Colombian restaurant', 
    'Cuban restaurant', 
    'Guatemalan restaurant', 
    'Honduran restaurant', 
    'Latin American restaurant', 
    'Nicaraguan restaurant', 
    'Peruvian restaurant', 
    'Salvadoran restaurant'], 'Latin American', inplace = True)

restaurants_cleaner['Food'].replace([
    'Bistro', 
    'French restaurant'], 'French', inplace = True)

restaurants_cleaner['Food'].replace([
    'Italian restaurant', 
    'Roman restaurant', 
    'Southern Italian restaurant'], 'Italian', inplace = True)

restaurants_cleaner['Food'].replace([
    'Pizza Delivery', 
    'Pizza restaurant'], 'Pizza', inplace = True)

restaurants_cleaner['Food'].replace([
    'Fish & chips restaurant', 
    'Seafood restaurant'], 'Seafood', inplace = True)

restaurants_cleaner['Food'].replace([
    'Fast food restaurant', 
    'Hot dog restaurant', 
    'Hot dog stand'], 'Fast Food', inplace = True)

restaurants_cleaner['Food'].replace([
    'Burrito restaurant', 
    'Mexican restaurant', 
    'Taco restaurant'], 'Mexican', inplace = True)

restaurants_cleaner['Food'].replace([
    'Cajun restaurant', 
    'Southern restaurant (US)', 
    'Barbecue restaurant'], 'Cajun/Southern', inplace = True)

restaurants_cleaner['Food'].replace([
    'Chicken restaurant', 
    'Chicken wings restaurant'], 'Chicken', inplace = True)

restaurants_cleaner['Food'].replace([
    'Deli', 
    'Sandwich shop'], 'Deli/Sandwiches', inplace = True)

restaurants_cleaner['Food'].replace([
    'Dinner theater', 
    'Eclectic restaurant', 
    'Pretzel store', 
    'Restaurant'], 'Other', inplace = True)

restaurants_cleaner['Food'].replace([
    'Sundanese restaurant', 
    'Eritrean restaurant'], 'African', inplace = True)

restaurants_cleaner['Food'].replace([
    'Gluten-free restaurant', 
    'Health food restaurant', 
    'Juice shop', 
    'Organic restaurant', 
    'Raw food restaurant', 
    'Salad shop', 
    'Vegan restaurant'], 'Healthy', inplace = True)

restaurants_cleaner['Food'].replace([
    'Greek restaurant', 
    'Israeli restaurant', 
    'Mediterranean restaurant'], 'Mediterranean', inplace = True)

restaurants_cleaner['Food'].replace('Hawaiian restaurant', 'Hawaiian', inplace = True)

restaurants_cleaner['Food'].replace([
    'Indian restaurant', 
    'Pakistani restaurant', 
    'Nepalese restaurant'], 'Indian/Himalayan', inplace = True)

restaurants_cleaner['Food'].replace('Fusion restaurant', 'Fusion', inplace = True)
 
restaurants_cleaner['Food'].replace([
    'Spanish restaurant', 
    'Tapas restaurant', 
    'Portuguese restaurant'], 'Spanish/Portuguese', inplace = True)

restaurants_cleaner['Food'].replace('Russian restaurant', 'Russian', inplace = True)

restaurants_cleaner['Food'].replace([
    'Diner', 
    'Hamburger restaurant'], 'Diner', inplace = True)

In [245]:
set(restaurants_cleaner['Food'])

{0,
 'African',
 'American',
 'Asian',
 'Bakery/Desserts',
 'Bar',
 'Breakfast/Brunch',
 'Cajun/Southern',
 'Chicken',
 'Chinese',
 'Coffee/Tea/Cafe',
 'Deli/Sandwiches',
 'Diner',
 'Fast Food',
 'French',
 'Fusion',
 'Hawaiian',
 'Healthy',
 'Indian/Himalayan',
 'Italian',
 'Japanese',
 'Korean',
 'Latin American',
 'Mediterranean',
 'Mexican',
 'Middle Eastern',
 'Other',
 'Pizza',
 'Russian',
 'Seafood',
 'Southeast Asian',
 'Spanish/Portuguese'}

In [246]:
set(restaurants_cleaner['Price'])

{'$', '$$', '$$$', '$$$$', 0}

In [247]:
restaurants_cleaner.head()

,Business Account Number,DBA Name,Street Address,Source Zipcode,Currently Open,Years Open,Year Opened,Month Opened,Year Closed,Success,Zip Code,Neighborhood,ZHVI_AllHomes,Year,Month,google_results,Rating,Price,Food
3540,1007473,The Pin Up,772 Folsom St,94108,1.0,4.350685,2015,1,None,1,94108.0,Chinatown,852600.0,2015.0,1.0,"(4.0, Diner, 0)",4.0,0,Diner
3585,1008176,Little Beijing Restaurant,1316 Noriega St,94122,1.0,4.279452,2015,1,None,1,94122.0,Sunset,986600.0,2015.0,1.0,"(4.1, Chinese restaurant, 0)",4.1,0,Chinese
3601,1008549,Wang Dae Bak,2642 Ocean Ave,94132,1.0,4.263014,2015,2,None,1,94132.0,Lake Merced,885900.0,2015.0,2.0,"(4.1, Korean restaurant, 0)",4.1,0,Korean
3615,1009049,Mateo's Taqueria,2471 Mission St,94110,1.0,4.238356,2015,2,None,1,94110.0,Inner Mission/Bernal Heights,1154200.0,2015.0,2.0,"(4.2, Mexican restaurant, 0)",4.2,0,Mexican
3622,477957,Ro Cafe,2739 Geary Blvd,94118,1.0,4.232877,2015,2,None,1,94118.0,Inner Richmond,1671600.0,2015.0,2.0,"(4.7, Middle Eastern restaurant, 0)",4.7,0,Middle Eastern


In [259]:
restaurants_clean = restaurants_cleaner[['Success', 'Month Opened', 'Neighborhood', 'Food', 'ZHVI_AllHomes']]

In [260]:
restaurants_clean.head()

,Success,Month Opened,Neighborhood,Food,ZHVI_AllHomes
3540,1,1,Chinatown,Diner,852600.0
3585,1,1,Sunset,Chinese,986600.0
3601,1,2,Lake Merced,Korean,885900.0
3615,1,2,Inner Mission/Bernal Heights,Mexican,1154200.0
3622,1,2,Inner Richmond,Middle Eastern,1671600.0


In [250]:
#Convert 'Neighborhood' to Columns
restaurants_dummies = pd.concat([restaurants_clean, pd.get_dummies(restaurants_clean['Neighborhood'])], axis=1)
restaurants_dummies.drop('Neighborhood', axis=1, inplace=True)


In [251]:
#Convert 'Food' to Columns
restaurants_dummies = pd.concat([restaurants_dummies, pd.get_dummies(restaurants_dummies['Food'])], axis=1)
restaurants_dummies.drop('Food', axis=1, inplace=True)

In [252]:
#Drop null '0' Column
restaurants_dummies.drop(columns=[0], inplace=True)

In [253]:
#Convert 'Month Opened' to strings with month names
restaurants_dummies['Month Opened']. replace(1, 'January', inplace = True)
restaurants_dummies['Month Opened']. replace(2, 'February', inplace = True)
restaurants_dummies['Month Opened']. replace(3, 'March', inplace = True)
restaurants_dummies['Month Opened']. replace(4, 'April', inplace = True)
restaurants_dummies['Month Opened']. replace(5, 'May', inplace = True)
restaurants_dummies['Month Opened']. replace(6, 'June', inplace = True)
restaurants_dummies['Month Opened']. replace(7, 'July', inplace = True)
restaurants_dummies['Month Opened']. replace(8, 'August', inplace = True)
restaurants_dummies['Month Opened']. replace(9, 'September', inplace = True)
restaurants_dummies['Month Opened']. replace(10, 'October', inplace = True)
restaurants_dummies['Month Opened']. replace(11, 'November', inplace = True)
restaurants_dummies['Month Opened']. replace(12, 'December', inplace = True)


In [254]:
#Convert 'Month Opened' to Columns
restaurants_dummies = pd.concat([restaurants_dummies, pd.get_dummies(restaurants_dummies['Month Opened'])], axis=1)
restaurants_dummies.drop('Month Opened', axis=1, inplace=True)

In [255]:
restaurants_dummies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1285 entries, 3540 to 5289
Data columns (total 65 columns):
Success                                    1285 non-null int64
Bayview-Hunters Point                      1285 non-null uint8
Castro/Noe Valley                          1285 non-null uint8
Chinatown                                  1285 non-null uint8
Haight-Ashbury                             1285 non-null uint8
Hayes Valley/Tenderloin/North of Market    1285 non-null uint8
Ingelside-Excelsior/Crocker-Amazon         1285 non-null uint8
Inner Mission/Bernal Heights               1285 non-null uint8
Inner Richmond                             1285 non-null uint8
Lake Merced                                1285 non-null uint8
Marina                                     1285 non-null uint8
North Beach/Chinatown                      1285 non-null uint8
Outer Richmond                             1285 non-null uint8
Parkside/Forest Hill                       1285 non-null uint8
Polk/Rus

In [261]:
restaurants_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1285 entries, 3540 to 5289
Data columns (total 5 columns):
Success          1285 non-null int64
Month Opened     1285 non-null int64
Neighborhood     1285 non-null object
Food             1285 non-null object
ZHVI_AllHomes    1285 non-null float64
dtypes: float64(1), int64(2), object(2)
memory usage: 60.2+ KB


In [262]:
restaurants_clean.to_pickle('./restaurants_clean.pkl')

In [258]:
restaurants_dummies.to_pickle('./restaurants_dummies.pkl')